In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
def read_merge(file1, file2):
    # read csv files
    os.chdir("/Users/jekaterina/Desktop/Bakalauras/Data")
    df1 = pd.read_csv(file1, sep=';')
    df2 = pd.read_csv(file2, sep=';')
    
    if(df1.shape[0] > df2.shape[0]):
        df1.drop(df1.tail(1).index, inplace=True)
    else:
        df2.drop(df2.tail(1).index, inplace=True)
    
    df = pd.merge(df1, df2, how='inner', on='Date')
    
    return df


def data_preparation(df):
    df["Date"]= pd.to_datetime(df["Date"]).dt.date
    
    if('Addresses with balance greater than $10M' in df.columns):
        df.rename(columns = {'Addresses with balance greater than $10M':'Addresses with balance > $10M',}, inplace=True)
        
    df['Price (Open)'] = df['Price (Open)'].str.replace(',','')
        
    for col in df.columns:
        df[col] = df[col].apply(lambda x: str(x).replace(",","")) 
        df[col] = df[col].apply(lambda x: str(x).replace("$","")) 
        #df[col] = df[col].apply(lambda x: str(x).replace("-",np.nan))
        df[col] = df[col].replace('-', np.NaN)
        
        for row in df[col]:
            
            if 'K' in str(row):
                df[col] = df[col].replace(row, float(row[:-1])*1000)
                
            if 'M' in str(row):
                df[col] = df[col].replace(row, float(row[:-1])*1000000)
                
            if 'B' in str(row):
                df[col] = df[col].replace(row, float(row[:-1])*1000000000)
                
            if 'T' in str(row):
                df[col] = df[col].replace(row, float(row[:-1])*1000000000000)
                
    return df


def save_csv(df, crypto):
    os.chdir("/Users/jekaterina/Desktop/Bakalauras/Data/Prepared data")
    df.to_csv(crypto + '_daily.csv', sep=';', index=False)

In [3]:
for crypto in ['BTC', 'ETH', 'BNB', 'XRP', 'ADA', 'DOGE', 'LTC',
               'TRX', 'LINK', 'BCH', 'EOS', 'OMG', 'XMR', 'ZEC']:
    df = read_merge(crypto + '_yearly_1.csv', crypto + '_yearly_2.csv')
    df = data_preparation(df)
    save_csv(df, crypto)